In [ ]:
!pip install pandas-gbq==0.21.0

  Attempting uninstall: pandas-gbq
    Found existing installation: pandas-gbq 0.26.1
    Uninstalling pandas-gbq-0.26.1:
      Successfully uninstalled pandas-gbq-0.26.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 1.34.0 requires pandas-gbq>=0.26.0, but you have pandas-gbq 0.21.0 which is incompatible.


In [1]:

"""
ETL PROFISSIONAL: Vendas de E-commerce (Simulador Nível #9)
Autor: Eric Pimentel
"""

import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from google.auth import default
from google.colab import auth
from google.cloud import bigquery
from googleapiclient.discovery import build
import datetime
import pandas_gbq
import gspread
import kagglehub
import os
import logging
import warnings
warnings.filterwarnings("ignore")

# Configurar logs
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


# 1. EXTRAÇÃO

def extract_data():
    """Extrai dados do Kaggle."""
    try:
        logging.info(" Iniciando extração...")
        path = kagglehub.dataset_download("adarsh0806/influencer-merchandise-sales")
        files = os.listdir(path)

        for file in files:
            if file.endswith(".csv"):
                df = pd.read_csv(
                    os.path.join(path, file),
                    delimiter=',',
                    parse_dates=['Order Date']
                )
                logging.info(" Dados extraídos com sucesso!")
                return df

        raise FileNotFoundError("Nenhum CSV encontrado.")

    except Exception as e:
        logging.error(f" Erro na extração: {e}")
        return None


# 2. TRANSFORMAÇÃO (COM REGRESSÃO LOGÍSTICA)

def transform_data(df):
    """Transforma dados e aplica modelo de previsão."""
    try:
        logging.info(" Iniciando transformação...")

        # ----- TRADUÇÃO DAS COLUNAS -----
        df = df.rename(columns={
            'Order ID': 'ID_Pedido',
            'Order Date': 'Data_Pedido',
            'Product Category': 'Categoria_Produto',
            'Buyer Gender': 'Genero_Comprador',
            'Buyer Age': 'Idade_Comprador',
            'Order Location': 'Localizacao_Pedido',
            'International Shipping': 'Envio_Internacional',
            'Sales Price': 'Preco_Venda',
            'Shipping Charges': 'Taxas_Envio',
            'Sales per Unit': 'Vendas_por_Unidade',
            'Quantity': 'Quantidade',
            'Total Sales': 'Vendas_Totais',
            'Rating': 'Avaliacao',
            'Review': 'Resenha'
        })

        # ----- LIMPEZA E ENGENHARIA DE FEATURES -----
        df = df.drop_duplicates(subset=['ID_Pedido'])
        df['Margem_Lucro'] = df['Vendas_Totais'] - (df['Preco_Venda'] * df['Quantidade'] + df['Taxas_Envio'])
        df['Faixa_Etaria'] = pd.cut(df['Idade_Comprador'], bins=[0, 18, 25, 35, 100], labels=['<18', '18-25', '26-35', '35+'])

        # ----- CORRIGIR COLUNA 'Envio_Internacional' -----
        # Converter "Yes"/"No" para 1/0
        df['Envio_Internacional'] = df['Envio_Internacional'].map({'Yes': 1, 'No': 0}).astype(int)

        # Verificar se há valores nulos ou inválidos
        if df['Envio_Internacional'].isnull().any():
            logging.warning("🚨 Valores inválidos encontrados em 'Envio_Internacional'. Substituindo por 0.")
            df['Envio_Internacional'] = df['Envio_Internacional'].fillna(0)

        # ----- PREPARAÇÃO PARA REGRESSÃO LOGÍSTICA -----
        # Agrupar dados por mês para criar variável alvo (y)
        df_mensal = df.groupby(pd.Grouper(key='Data_Pedido', freq='M')).agg({
            'Vendas_Totais': 'sum',
            'Avaliacao': 'mean',
            'Envio_Internacional': 'sum'
        }).reset_index()

        # Criar variável alvo: 1 se as vendas do próximo mês forem maiores, 0 caso contrário
        df_mensal['Target'] = (df_mensal['Vendas_Totais'].shift(-1) > df_mensal['Vendas_Totais']).astype(int)
        df_mensal = df_mensal.dropna()

        # Definir features (X) e target (y)
        X = df_mensal[['Vendas_Totais', 'Avaliacao', 'Envio_Internacional']]
        y = df_mensal['Target']

        # Treinar modelo
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        model = LogisticRegression()
        model.fit(X_train, y_train)

        # Fazer previsões
        df_mensal['Previsao_Vendas'] = model.predict(X)
        logging.info(f" Modelo treinado! Acurácia: {accuracy_score(y_test, model.predict(X_test)):.2%}")

        # Mesclar previsões com o dataset original
        df_mensal['Mes'] = df_mensal['Data_Pedido'].dt.strftime('%Y-%m')
        df['Mes'] = df['Data_Pedido'].dt.strftime('%Y-%m')
        df = pd.merge(df, df_mensal[['Mes', 'Previsao_Vendas']], on='Mes', how='left')

        logging.info(" Dados transformados e previsões geradas!")
        return df

    except Exception as e:
        logging.error(f" Erro na transformação: {e}")
        return None


# 3. CARREGAMENTO (ATUALIZADO COM PREVISÕES)

def load_data(df):
    """Carrega dados para Google Sheets e BigQuery."""
    try:
        logging.info(" Iniciando carregamento...")
        auth.authenticate_user()
        credentials, _ = default()
        gc = gspread.authorize(credentials)

        # Converter a coluna 'Data_Pedido' para string antes de carregar
        df['Data_Pedido'] = df['Data_Pedido'].dt.strftime('%Y-%m-%d %H:%M:%S')

        # Google Sheets
        spreadsheet = gc.create('Vendas_Tratadas_Influenciador02')
        spreadsheet.share(None, perm_type='anyone', role='writer')
        worksheet = spreadsheet.get_worksheet(0)
        worksheet.update([df.columns.tolist()] + df.values.tolist())

        # Obter a URL da planilha
        spreadsheet_url = spreadsheet.url

        # Exibir a URL
        print(f"URL da planilha: {spreadsheet_url}")

        # BigQuery
        client = bigquery.Client(project='projeto-vendas-influenciador')
        pandas_gbq.to_gbq(
            df,
            destination_table='projeto_vendas.vendas_tratadas02',
            project_id=client.project,
            if_exists='replace'
        )
        logging.info(" Dados no BigQuery: Prontos para SQL avançado!")

        return True

    except Exception as e:
        logging.error(f" Erro no carregamento: {e}")
        return False


# 4. EXECUÇÃO PRINCIPAL

if __name__ == "__main__":
    dados_brutos = extract_data()
    if dados_brutos is not None:
        dados_tratados = transform_data(dados_brutos)
        if dados_tratados is not None:
            sucesso = load_data(dados_tratados)
            if sucesso:
                logging.info(" Missão cumprida! Previsões no Looker!")
            else:
                logging.warning(" Concluído com erros parciais.")

100%|██████████| 122k/122k [00:00<00:00, 45.9MB/s]

Extracting files...


URL da planilha: https://docs.google.com/spreadsheets/d/1K9Px4FfEiiIUN3hbZo8BQZRVU_i2LSig1qRVimS7hno


100%|██████████| 1/1 [00:00<00:00, 702.33it/s]
